# Experiment 1

**Target Task:** Cora 

**Transfer learning Source Task:** Citeseer

**Meta-learning Source Task:** Citeseer, Pubmed
***
## Installs & imports

In [ ]:
# ! pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# ! pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
# ! pip install torch-geometric

# ! pip install --ignore-installed "jsonschema>=2.6.0,<3.1.0"
! pip install comet_ml --upgrade --ignore-installed

In [7]:
import numpy as np
import copy
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, VGAE
from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid
from tqdm import tqdm
from comet_ml import Experiment

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

***
## Data & Experiment Setup

In [2]:
cora = Planetoid('./data', 'Cora')
cora[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [11]:
experiment = Experiment(project_name='experiment-1')

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/graph-net-experiments/experiment-1/84afda89c97c4b43969df67ce76e5b6b
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     os packages         : 1
COMET INFO: ---------------------------
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/graph-net-experiments/experiment-1/5e868795131b455fb7e4471a0dedf037



***
## Training base models on Cora

We use 2 layer networks...

### GCN

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)